### TODO 
1. Test for all countries and add country specific attributes. 
2. Ablation study with **pd.fillna()** and **pd.notna()**. 
3. Ablation study with and w/o growth_rate. 

### Common libraries 

In [1]:
import torch.nn.functional as F
import torch 
import torch.optim as optim
import sys
import torch.nn as nn
import pandas as pd 
import os
import numpy as np
from dateutil.parser import parse 

def dateConvertor(date):
    dt = parse(date)
    date = dt.strftime('%Y-%m-%d')
    return date

### Reading data (for all country)
TODO: 
1. Data cleaning has to be done efficiently. In following code, simply removing Jan, Feb and Dec data and linearly interpolating all NAN. 
2. If we don't have history, then following program will exit (sys.exit call). Replace this with try and catch.

In [2]:
country_codes = ['ABW','AFG','AGO','ALB','AND','ARE','ARG','AUS','AUT','AZE','BDI','BEL','BEN','BFA','BGD','BGR','BHR','BHS','BIH','BLR','BLZ','BMU','BOL','BRA','BRB','BRN','BTN','BWA','CAF','CAN','CHE','CHL','CHN','CIV','CMR','COD','COG','COL','COM','CPV','CRI','CUB','CYP','CZE','DEU','DJI','DMA','DNK','DOM','DZA','ECU','EGY','ERI','ESP','EST','ETH','FIN','FJI','FRA','FRO','GAB','GBR','GEO','GHA','GIN','GMB','GRC','GRL','GTM','GUM','GUY','HKG','HND','HRV','HTI','HUN','IDN','IND','IRL','IRN','IRQ','ISL','ISR','ITA','JAM','JOR','JPN','KAZ','KEN','KGZ','KHM','KOR','KWT','LAO','LBN','LBR','LBY','LKA','LSO','LTU','LUX','LVA','MAC','MAR','MCO','MDA','MDG','MEX','MLI','MMR','MNG','MOZ','MRT','MUS','MWI','MYS','NAM','NER','NGA','NIC','NLD','NOR','NPL','NZL','OMN','PAK','PAN','PER','PHL','PNG','POL','PRI','PRT','PRY','PSE','QAT','RKS','ROU','RUS','RWA','SAU','SDN','SEN','SGP','SLB','SLE','SLV','SMR','SOM','SRB','SSD','SUR','SVK','SVN','SWE','SWZ','SYC','SYR','TCD','TGO','THA','TJK','TKM','TLS','TTO','TUN','TUR','TWN','TZA','UGA','UKR','URY','USA','UZB','VEN','VIR','VNM','VUT','YEM','ZAF','ZMB','ZWE']
filenames = ["c1_school_closing.csv", "c2_workplace_closing.csv", "c3_cancel_public_events.csv", "c4_restrictions_on_gatherings.csv", "c5_close_public_transport.csv", "c6_stay_at_home_requirements.csv", "c7_movementrestrictions.csv", "c8_internationaltravel.csv", "confirmed_cases.csv"]

country_code2id = {}
for i in range(len(country_codes)):
    country_code2id[country_codes[i]] = i 

# date extraction
npi_date = pd.DataFrame({})
npi_df = pd.read_csv(os.path.join('timeseries', filenames[0])).T
npi_date['Date'] = npi_df.index.values[3:]
npi_date['Date'] = npi_date['Date'].apply(dateConvertor)

# extract data 
dataframes = {} 
countries_to_extract = ['ITA','IND','USA','CHN','GEO'] # countries code for which you want data. 
index = [country_code2id[code] for code in countries_to_extract]
for file in filenames:
    npi_df = pd.read_csv(os.path.join('timeseries', file)).T[3:]
    npi_df['Date'] = npi_date['Date'].values
    npi_df.set_index('Date', drop=True, inplace=True)
    npi_df = npi_df[index] # selecting countries 
    npi_df = npi_df[64:335] # removing Jan, Feb and Dec data
    for col in npi_df:
        npi_df[col] = pd.to_numeric(npi_df[col], errors='coerce') # converting object to numeric 
    npi_df.interpolate(method='linear', inplace=True) # interpolate missing values 
    dataframes[file[:-4]] = npi_df
    
    # computing growth rate
    if(file[:-4]=='confirmed_cases'):
        npi_df = pd.read_csv(os.path.join('timeseries', file)).T[3:]
        npi_df['Date'] = npi_date['Date'].values
        npi_df.set_index('Date', drop=True, inplace=True)
        npi_df = npi_df[index]
        npi_df = npi_df[64:335]
        for col in npi_df:
            npi_df[col] = pd.to_numeric(npi_df[col], errors='coerce')
        npi_df = 100*npi_df.diff()/npi_df
        npi_df.interpolate(method='linear', inplace=True) # interpolate missing values     
        dataframes['growth_rate'] = npi_df#.rolling(3).mean()

def readData(attributes, history, date):
    index = dataframes['c1_school_closing'].index.get_loc(date)
    if(history>index):
        print('Not sufficient history')
        sys.exit()
    data = []
    for att in attributes:
        temp = dataframes[att].iloc[index-history:index].values
        if(len(data)==0):
            data = np.asarray(temp)
        else:
            data = np.dstack((data, temp))
    x = torch.from_numpy(data).to(dtype=torch.double).permute(1,0,2)
    y = torch.from_numpy(dataframes['growth_rate'].iloc[index].values).to(dtype=torch.double)
    return x,y

### Reading data (for single country)

In [ ]:
filenames = ["c1_school_closing.csv", "c2_workplace_closing.csv", "c3_cancel_public_events.csv",
            "c4_restrictions_on_gatherings.csv", "c5_close_public_transport.csv", "c6_stay_at_home_requirements.csv", "c7_movementrestrictions.csv",
            "c8_internationaltravel.csv", "confirmed_cases.csv"]
# filenames = np.core.defchararray.add('timeseries/', np.asarray(filenames))
npi_data = pd.DataFrame({})

# date extraction 
file = filenames[0]
npi_df = pd.read_csv(os.path.join('timeseries', file))
npi_df = npi_df[npi_df['country_name']=='India'].iloc[:,3:].T
npi_data['Date'] = npi_df[77].index.values
npi_data['Index'] = npi_data['Date'].index.values/100.0

# other attributes extraction 
for file in filenames:
    npi_df = pd.read_csv(os.path.join('timeseries', file))
    npi_df = npi_df[npi_df['country_name']=='India'].iloc[:,3:].T
    npi_data[file[:-4]] = npi_df[77].values

# compute growth rate 
npi_data['growth_rate'] = npi_data['confirmed_cases'].diff()
npi_data['growth_rate'] = 100*npi_data['growth_rate']/npi_data['confirmed_cases'] # (0,1) or (0,100)

# smoothing growth_rate
npi_data['growth_rate'] = npi_data['growth_rate'].rolling(3).mean()

# cleaning df
npi_data = npi_data[64:]
# for col in npi_data.columns:
#     npi_data= npi_data[npi_data[col].notna()]

# interpolating instead of skipping 
npi_data.interpolate(method='linear', inplace=True)

### Linear model

In [80]:
class LinearModel(torch.nn.Module):
    def __init__(self, in_=300, out_=1):
        super(LinearModel, self).__init__()
        self.linear1 = torch.nn.Linear(in_, 64)
        self.linear2 = torch.nn.Linear(64, 8)
        self.linear3 = torch.nn.Linear(8, 1)
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = x.reshape(x.shape[0],-1)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        x = self.linear3(x)
        return x

class MultiHeadSharedBB(torch.nn.Module):
    def __init__(self, in_=300, out_=1):
        super(MultiHeadSharedBB, self).__init__()
        self.linear1 = torch.nn.Linear(in_, 64)
        self.linear2 = torch.nn.Linear(64, 8)
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = x.reshape(x.shape[0],-1)
        x = self.relu(self.linear1(x))
        x = self.relu(self.linear2(x))
        return x

class Heads(torch.nn.Module):
    def __init__(self):
        super(Heads, self).__init__()
        self.linear = torch.nn.Linear(8, 1)
        
    def forward(self, x):
        x = self.linear(x)
        return x

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

### Training multi-head model

In [96]:
country_count = len(countries_to_extract)
training_attributes = ["c1_school_closing", "c2_workplace_closing", "c3_cancel_public_events", "c4_restrictions_on_gatherings", "c5_close_public_transport", "c6_stay_at_home_requirements", "c7_movementrestrictions", "c8_internationaltravel", "confirmed_cases"]
history = 30
train_dates = npi_date['Date'][95:300].values
test_dates = npi_date['Date'][300:335].values

model = MultiHeadSharedBB(len(training_attributes)*history).to(dtype=torch.double)
model.apply(init_weights)
heads = []
optimizers = []

for i in range(country_count+1):
    head = Heads().to(dtype=torch.double)
    head.apply(init_weights)
    optimizer = optim.Adam(list(model.parameters())+list(head.parameters()), lr=1e-4, weight_decay=1e-5)
    optimizers.append(optimizer)
    heads.append(head)
mse_loss = torch.nn.MSELoss()

def validate(model, heads):
    model.eval()
    for head in heads:
        head.eval()
    
    validation_loss = [0]*len(heads)
    for i in range(len(test_dates)):
        x,y = readData(attributes=training_attributes, history=history, date=test_dates[i])
        features = model(x)
        y_pred = heads[0](features)
        loss = mse_loss(y_pred[:,0],y)
        validation_loss[0] += loss.detach().numpy()
        for i in range(len(heads)-1):
            y_pred = heads[i+1](features[i,:])
            loss = mse_loss(y_pred[0],y[i])
            validation_loss[i+1] += loss.detach().numpy()
    validation_loss[0]/=country_count
    model.train()
    for head in heads:
        head.train()
    return validation_loss

print('validation loss before training', validate(model,heads))

for epoch in range(200):
    np.random.shuffle(train_dates)
    for i in range(len(train_dates)):
        for optimizer in optimizers:
            optimizer.zero_grad()
        x,y = readData(attributes=training_attributes, history=history, date=train_dates[i])
        features = model(x)
        y_pred = heads[0](features)
        loss = mse_loss(y_pred[:,0],y)
        loss.backward(retain_graph=True)
        for i in range(len(heads)-1):
            y_pred = heads[i+1](features[i,:])
            loss = mse_loss(y_pred[0],y[i])
            loss.backward(retain_graph=True)
        for optimizer in optimizers:
            optimizer.step()
    print('epoch %d'%(epoch), ' loss ', validate(model, heads))


validation loss before training [8394984581291.745, 7049241325069.439, 123517424808697.19, 526010558587857.56, 64246272089.61998, 789556002.5321639]
epoch 0  loss  [174774554289.29507, 83403573493.2931, 5761709303415.072, 298276911622.3877, 12965318284.412048, 247878070.52428573]
epoch 1  loss  [26711831968.932163, 16084880682.376396, 527054232814.8067, 9375284083.421244, 3660059339.5834455, 59035692.64504522]
epoch 2  loss  [1689506151.5145974, 11078217841.365812, 1144699156654.3545, 541056360768.1372, 1435353619.7302754, 9247623.306264393]
epoch 3  loss  [41.82411556760019, 352.54571882392406, 8.43997162721236, 57.101693554787005, 0.13466124369910185, 622.782322330971]
epoch 4  loss  [41.82394255592773, 352.54416118125107, 8.44047012958048, 57.100750159917084, 0.1347078717162034, 622.6365737788724]
epoch 5  loss  [41.82390078905756, 352.54387323006097, 8.440437004168274, 57.10070445903878, 0.13470736383379567, 622.4670204316197]
epoch 6  loss  [41.82385338186289, 352.543548364339, 8.

KeyboardInterrupt: 

### Training single head model 

In [97]:
training_attributes = ["c1_school_closing", "c2_workplace_closing", "c3_cancel_public_events", "c4_restrictions_on_gatherings", "c5_close_public_transport", "c6_stay_at_home_requirements", "c7_movementrestrictions", "c8_internationaltravel", "confirmed_cases"]
history = 30 
model = LinearModel(len(training_attributes)*history).to(dtype=torch.double)
model.apply(init_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
mse_loss = torch.nn.MSELoss(reduction='none')

train_dates = npi_date['Date'][95:300].values
test_dates = npi_date['Date'][300:335].values

def validate(model):
    model.eval()
    validation_loss = [0]*len(countries_to_extract)
    for i in range(len(test_dates)):
        x,y = readData(attributes=training_attributes, history=history, date=test_dates[i])
        y_pred = model(x)
        loss = mse_loss(y_pred[:,0], y)
        validation_loss += loss.detach().numpy()
    model.train()
    return validation_loss
print('validation loss before training', validate(model))

model.train()
for epoch in range(200):
    np.random.shuffle(train_dates)
    for i in range(len(train_dates)):
        x,y = readData(attributes=training_attributes, history=history, date=train_dates[i])
        y_pred = model(x)
        loss = mse_loss(y_pred[:,0], y).mean()
        loss.backward() # computing gradients 
        optimizer.step() # updating weights 
    print('epoch %d validation loss ' %(epoch+1), validate(model))

validation loss before training [1.77959711e+12 2.94203879e+14 3.58035632e+14 3.71653081e+10
 4.91911585e+09]
epoch 1 validation loss  [2.38748107e+12 3.59885121e+14 4.74178704e+14 4.35988279e+10
 7.40116605e+09]
epoch 2 validation loss  [3.18411103e+12 4.92325878e+14 6.30368122e+14 5.97293953e+10
 1.08080625e+10]
epoch 3 validation loss  [1.42395950e+12 1.95894923e+14 2.51242583e+14 2.32570911e+10
 5.30639858e+09]
epoch 4 validation loss  [3.49371427e+02 6.97004001e+00 5.21798790e+01 4.64977569e-02
 6.17149268e+02]
epoch 5 validation loss  [3.43271040e+02 6.09526407e+00 4.97464981e+01 1.46565638e-01
 6.08685465e+02]
epoch 6 validation loss  [3.37957177e+02 5.37603820e+00 4.76568633e+01 2.84474664e-01
 6.01293559e+02]
epoch 7 validation loss  [3.33042284e+02 4.74722788e+00 4.57496742e+01 4.55238104e-01
 5.94440175e+02]
epoch 8 validation loss  [328.37398491   4.18319421  43.96149109   0.65685999 587.91561604]
epoch 9 validation loss  [323.87583738   3.67108898  42.260498     0.88835561

KeyboardInterrupt: 

### Training baseline model without positional info

1. Add country specific info here while creating dataset for model. 

In [ ]:
history = 30

y_total = npi_data['growth_rate'].values
x_total = npi_data.drop(columns=['Date', 'confirmed_cases']).to_numpy(dtype=np.float)

x_total = torch.from_numpy(x_total).to(dtype=torch.double)
y_total = torch.from_numpy(y_total).to(dtype=torch.double)

model = LinearModel(x_total.shape[1]*history).to(dtype=torch.double)
model.apply(init_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
mse_loss = torch.nn.MSELoss()

split_index = 150

x_train = torch.clone(x_total[0:split_index,:])
y_train = torch.clone(y_total[0:split_index])
x_test = torch.clone(x_total[split_index-history:,:])
y_test = torch.clone(y_total[split_index-history:])

print('attributes | ', x_train.shape[1])
print('training | ', len(x_train))
print('test | ', len(x_test)-history)
print('-'*20)

model.eval()

validation_loss = 0.0
x_temp = x_test[0:history,:]
y_pred = model(x_temp)
loss = mse_loss(y_pred, y_test[1])
validation_loss += loss.item()
for i in range(history+1,len(x_test)-1):
    x_temp = x_test[i-history:i,:] # [30,9] --> [1,30*9]
    x_temp[-1,-1] = y_pred.item()
    y_pred = model(x_temp)
    loss = mse_loss(y_pred, y_test[i+1])
    validation_loss += loss.item()
print('validation loss before training %0.2f' %(validation_loss))

model.train()
for epoch in range(200):
    training_loss = 0.0
    x_train = torch.clone(x_total[0:split_index,:])
    y_train = torch.clone(y_total[0:split_index])
    for i in range(history,len(x_train)-1):
        optimizer.zero_grad() # make gradients zero 
        x_temp = x_train[i-history:i,:]
        y_pred = model(x_temp)
        loss = mse_loss(y_pred, y_train[i+1])
        loss.backward() # computing gradients 
        optimizer.step() # updating weights 
        training_loss += loss.item() 
    if((epoch+1)%5 == 0):
        model.eval()
        validation_loss = 0.0
        x_test = torch.clone(x_total[split_index-history:,:])
        y_test = torch.clone(y_total[split_index-history:])
        
        x_temp = x_test[0:history,:]
        y_pred = model(x_temp)
        loss = mse_loss(y_pred, y_test[1])
        validation_loss += loss.item()
        for i in range(history+1,len(x_test)-1):
            x_temp = x_test[i-history:i,:]
            x_temp[-1,-1] = y_pred.item()
            y_pred = model(x_temp)
            loss = mse_loss(y_pred, y_test[i+1])
            validation_loss += loss.item()
        print('epoch %d | training loss %0.2f | validation loss %0.2f'%(epoch, training_loss, validation_loss))
        model.train()        